In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False)




In [23]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False, header=False)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [2]:
x_train = np.load("/home/jovyan/UNSW/x_train.npy")
y_train = np.load("/home/jovyan/UNSW/y_train.npy")
x_val = np.load("/home/jovyan/UNSW/x_val.npy")
y_val = np.load("/home/jovyan/UNSW/y_val.npy")
x_test = np.load("/home/jovyan/UNSW/x_test.npy")
y_test = np.load("/home/jovyan/UNSW/y_test.npy")

In [26]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [27]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [28]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/dnn_pytorch.pt"))

/tmp/ipykernel_1072/3756506766.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/UNSW/transfer_attack/dnn_pytorch.pt"))


<All keys matched successfully>

In [29]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [51]:
from art.attacks.evasion import DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [14]:
for epsilon in epsilon_values:
    adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [15]:
for epsilon in epsilon_values:
    adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False, max_iter=10) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [ ]:
for epsilon in epsilon_values:
    adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False, max_iter=10) # init eps=1e-06
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)


In [ ]:
from art.attacks.evasion import AutoProjectedGradientDescent

for epsilon in epsilon_values:
    adv_crafter = AutoProjectedGradientDescent(classifier, eps=epsilon, max_iter = 10, batch_size=128, verbose = False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [16]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

Accuracy: 0.8854357687481903

macro
Precision: 0.876212899139358
Recall: 0.8556184012066366
F1 Score: 0.8646819154737567

weighted
Precision: 0.884107669388339
Recall: 0.8854357687481903
F1 Score: 0.8838255479772675

Mean FNR: 0.14438159879336349
Mean TNR: 0.8556184012066366
Mean FPR: 0.14438159879336349
Mean TPR: 0.8556184012066366
Accuracy: 0.5536583867826026

macro
Precision: 0.528526268267531
Recall: 0.5321427512490862
F1 Score: 0.5234451561462188

weighted
Precision: 0.5923069778393741
Recall: 0.5536583867826026
F1 Score: 0.566791461283709

Mean FNR: 0.46785724875091395
Mean TNR: 0.5321427512490862
Mean FPR: 0.46785724875091395
Mean TPR: 0.5321427512490862
Accuracy: 0.052241359644119995

macro
Precision: 0.06917943095793198
Recall: 0.04271562504028567
F1 Score: 0.050582490899224564

weighted
Precision: 0.09121771222269683
Recall: 0.052241359644119995
F1 Score: 0.06491861287573379

Mean FNR: 0.9572843749597143
Mean TNR: 0.04271562504028567
Mean FPR: 0.9572843749597143
Mean TPR: 0.0

In [18]:
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

Accuracy: 0.8853655754534048

macro
Precision: 0.8761216536902561
Recall: 0.8555449638764119
F1 Score: 0.8646012044386691

weighted
Precision: 0.8840351026934499
Recall: 0.8853655754534048
F1 Score: 0.88375539692495

Mean FNR: 0.14445503612358818
Mean TNR: 0.8555449638764119
Mean FPR: 0.14445503612358818
Mean TPR: 0.8555449638764119
Accuracy: 0.4996007756359074

macro
Precision: 0.4924367431498884
Recall: 0.4913152904159691
F1 Score: 0.4786596071230793

weighted
Precision: 0.5577924669437923
Recall: 0.4996007756359074
F1 Score: 0.5164045002515989

Mean FNR: 0.5086847095840308
Mean TNR: 0.4913152904159691
Mean FPR: 0.5086847095840308
Mean TPR: 0.4913152904159691
Accuracy: 0.019469865141132393

macro
Precision: 0.027233470540432316
Recall: 0.01646103017143741
F1 Score: 0.019270000555163116

weighted
Precision: 0.035663340991538076
Recall: 0.019469865141132393
F1 Score: 0.024327545035151502

Mean FNR: 0.9835389698285626
Mean TNR: 0.01646103017143741
Mean FPR: 0.9835389698285626
Mean TPR: 

In [31]:
import sys
sys.path.append('/home/jovyan/adversarial-attacks-pytorch')


In [32]:
from torchattacks import VNIFGSM

epsilon_values = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    attack = VNIFGSM(model, eps=epsilon, alpha=0.02, steps=10, decay=1.0, N=5, beta=1.5)
    x_test_tensor = torch.from_numpy(x_test).float()
    y_test_tensor = torch.from_numpy(y_test).long()
    x_test_adv = attack(x_test_tensor, y_test_tensor)
    x_test_adv = x_test_adv.cpu().numpy()

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_VNIFGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)



In [33]:
print("start VNIFGSM")

# y_test_BA = np.load("/home/jovyan/UNSW/y_test_BA.npy")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_VNIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'VNIFGSM', epsilon)

start VNIFGSM
Accuracy: 0.8858481543550553

macro
Precision: 0.8766589998374265
Recall: 0.8561619378191776
F1 Score: 0.8651897939740362

weighted
Precision: 0.8845311295247961
Recall: 0.8858481543550553
F1 Score: 0.8842534576577061

Mean FNR: 0.14383806218082235
Mean TNR: 0.8561619378191776
Mean FPR: 0.14383806218082235
Mean TPR: 0.8561619378191776
Accuracy: 0.530661308578498

macro
Precision: 0.5159838628264448
Recall: 0.5182595506973335
F1 Score: 0.5064355187083666

weighted
Precision: 0.5807587710757517
Recall: 0.530661308578498
F1 Score: 0.5459365085991857

Mean FNR: 0.4817404493026665
Mean TNR: 0.5182595506973335
Mean FPR: 0.4817404493026665
Mean TPR: 0.5182595506973335
Accuracy: 0.08321415096822876

macro
Precision: 0.10377779214575462
Recall: 0.06651152308392128
F1 Score: 0.07866556176640613

weighted
Precision: 0.13743414956654243
Recall: 0.08321415096822876
F1 Score: 0.10205092215954294

Mean FNR: 0.9334884769160787
Mean TNR: 0.06651152308392128
Mean FPR: 0.9334884769160787
Me

In [34]:
from torchattacks import SINIFGSM

epsilon_values = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    attack = SINIFGSM(model, eps=epsilon, alpha=0.03, steps=10, decay=1.0, m=5)
    x_test_tensor = torch.from_numpy(x_test).float()
    y_test_tensor = torch.from_numpy(y_test).long()

    x_test_adv = attack(x_test_tensor, y_test_tensor)
    x_test_adv = x_test_adv.cpu().numpy()

    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_SINIFGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)



In [35]:
print("start SINIFGSM")


for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_SINIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)
    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'SINIFGSM', epsilon)

start SINIFGSM
Accuracy: 0.9014222916355915

macro
Precision: 0.8930158537891857
Recall: 0.8771172446624935
F1 Score: 0.8843584551191599

weighted
Precision: 0.9004767055552159
Recall: 0.9014222916355915
F1 Score: 0.9004053992310376

Mean FNR: 0.12288275533750648
Mean TNR: 0.8771172446624935
Mean FPR: 0.12288275533750648
Mean TPR: 0.8771172446624935
Accuracy: 0.5978275175263883

macro
Precision: 0.5761902135268435
Recall: 0.5862215237837409
F1 Score: 0.5723912290864396

weighted
Precision: 0.63795471764979
Recall: 0.5978275175263883
F1 Score: 0.6100657058775008

Mean FNR: 0.413778476216259
Mean TNR: 0.5862215237837409
Mean FPR: 0.413778476216259
Mean TPR: 0.5862215237837409
Accuracy: 0.05325916241850997

macro
Precision: 0.0706907975533026
Recall: 0.04245723042668744
F1 Score: 0.05131757049513569

weighted
Precision: 0.09394747632236666
Recall: 0.05325916241850997
F1 Score: 0.06682131003688173

Mean FNR: 0.9575427695733125
Mean TNR: 0.04245723042668744
Mean FPR: 0.9575427695733125
Mean

In [11]:
import sys
sys.path.append('/home/jovyan/attack-tabular')


In [12]:
class FixedPyTorchClassifier(PyTorchClassifier):
    def loss_gradient(self, x, y, training_mode=False, **kwargs):
        if not isinstance(y, torch.Tensor):
            y = torch.from_numpy(y).to(dtype=torch.long)
        elif y.dtype != torch.long:
            y = y.to(dtype=torch.long)
        
        x = torch.from_numpy(x).to(self._device).requires_grad_(True)
        y = y.to(self._device)
        
        self._model.train(training_mode)
        model_outputs = self._model(x)
        loss = self._loss(model_outputs[-1], y)
        
        self._model.zero_grad()
        loss.backward()
        
        if x.grad is None:
            raise ValueError("Gradient computation failed: x.grad is None")
        grad = x.grad.detach().cpu().numpy()
        return grad

In [15]:
import numpy as np
import torch
from art.estimators.classification import PyTorchClassifier
from src.attacks.cafa import CaFA


classifierCaFA = FixedPyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)


def detect_feature_types(data, unique_threshold=10):
    n_features = data.shape[1]
    cat_indices, ordinal_indices, cont_indices = [], [], []
    
    for i in range(n_features):
        unique_values = np.unique(data[:, i])
        n_unique = len(unique_values)
        is_float = np.issubdtype(data[:, i].dtype, np.floating)
        
        if n_unique <= unique_threshold:
            if is_float or not np.all(unique_values == unique_values.astype(int)):
                cat_indices.append(i)
            else:
                ordinal_indices.append(i)
        else:
            cont_indices.append(i)
    

    return (np.array(cat_indices, dtype=np.int64),
            np.array(ordinal_indices, dtype=np.int64),
            np.array(cont_indices, dtype=np.int64))

cat_indices, ordinal_indices, cont_indices = detect_feature_types(x_test)
print("Categorical indices:", cat_indices)
print("Ordinal indices:", ordinal_indices)
print("Continuous indices:", cont_indices)

feature_ranges = np.vstack([x_test.min(axis=0), x_test.max(axis=0)]).T
standard_factors = np.ones(x_test.shape[1])
one_hot_groups = []
if len(cat_indices) > 0:
    start_idx = cat_indices[0]
    group = [start_idx]
    for idx in cat_indices[1:]:
        if idx == group[-1] + 1:
            group.append(idx)
        else:
            one_hot_groups.append(np.array(group))
            group = [idx]
    if group:
        one_hot_groups.append(np.array(group))
print("One-hot groups:", [g.tolist() for g in one_hot_groups])

x_test = x_test.astype(np.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).numpy()




Categorical indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 24 33 36 42 45 50 51
 52 55]
Ordinal indices: []
Continuous indices: [17 18 19 20 21 22 23 25 26 27 28 29 30 31 32 34 35 37 38 39 40 41 43 44
 46 47 48 49 53 54]
One-hot groups: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [24], [33], [36], [42], [45], [50, 51, 52], [55]]


In [19]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    attack = CaFA(
        estimator=classifierCaFA,
        cat_indices=cat_indices,
        ordinal_indices=ordinal_indices,
        cont_indices=cont_indices,
        feature_ranges=feature_ranges,
        standard_factors=standard_factors,
        cat_encoding_method="one_hot_encoding",
        one_hot_groups=one_hot_groups,
        random_init=True,
        random_seed=42,
        max_iter=15,
        max_iter_tabpgd=25,
        eps=epsilon,
        step_size=0.001,
        perturb_categorical_each_steps=5,
    )
    
    x_test_adv = attack.generate(x=x_test, y=y_test_tensor)
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_CaFA_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

100%|██████████| 25/25 [00:18<00:00,  1.37it/s]


In [20]:
print("start CaFa")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_CaFA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)
    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'CaFA', epsilon)

start CaFa
Accuracy: 0.42184415333725245

macro
Precision: 0.2914073381132689
Recall: 0.31277650935456364
F1 Score: 0.30161557212150353

weighted
Precision: 0.3919637302036467
Recall: 0.42184415333725245
F1 Score: 0.4062918307332782

Mean FNR: 0.6872234906454364
Mean TNR: 0.31277650935456364
Mean FPR: 0.6872234906454364
Mean TPR: 0.31277650935456364
Accuracy: 0.4018566126470769

macro
Precision: 0.28488468589154275
Recall: 0.2980713153111343
F1 Score: 0.2912931242211057

weighted
Precision: 0.3834469583880552
Recall: 0.4018566126470769
F1 Score: 0.3924129345548905

Mean FNR: 0.7019286846888657
Mean TNR: 0.2980713153111343
Mean FPR: 0.7019286846888657
Mean TPR: 0.2980713153111343
Accuracy: 0.3901869773889849

macro
Precision: 0.28102106227106227
Recall: 0.2895203956007123
F1 Score: 0.28519273064453915

weighted
Precision: 0.3783271284625796
Recall: 0.39018697738898483
F1 Score: 0.3841560578587236

Mean FNR: 0.7104796043992877
Mean TNR: 0.2895203956007123
Mean FPR: 0.7104796043992877
Mea